# Projekt PSI
Analiza 5: co jest silniejsze w niespójnym warunku (inc): dźwięk vs obraz? 
a może negatywne vs pozytywne? 
(np. nieważne czy negatywny jest dźwięk czy obraz, ważne że co najmniej jedno jest negatywne i całość też jest negatywna)


## Loading data


In [25]:
%matplotlib inline
import os
import pandas
import matplotlib.pyplot as plt
import seaborn

# files that we use, additional windows path handling for windows douche bags
if os.name == 'nt':
    bitalino_filename = '..\\data\\bitalino\\1107_opensignals_prawestanowisko_2019-04-19_09-11-49.txt'
    IADS2_filename = '..\\data\\IADS2.csv'
    df_filename = "..\\data\\procedura\\1107_2019_Apr_19_0712.txt"
    IAPS_filename = '..\\data\\IAPS.csv'
else:
    bitalino_filename = '../data/2019-afcai-spring/bitalino/1107_opensignals_prawestanowisko_2019-04-19_09-11-49.txt'
    IADS2_filename = '../data/IADS2.csv'
    df_filename = "../data/procedura/1107_2019_Apr_19_0712.txt"
    IAPS_filename = '../data/IAPS.csv'

print("Filenames Loaded")

Filenames Loaded


Ten sposób jakiś upośledzony jest, można to jakoś mądrzej zrobić. A i w sumie to nam to chyba nie potrzebne xd
ODP: na razie to nie będzie potrzebne, tym można się później zajać jako dodatkowa 
rzecz/rozwinięcie projektu (красный лиса и зелёный лиса, приветствие! который объясняет этого идиота.)

In [26]:
hr = pandas.read_csv(bitalino_filename,
                     skiprows=[0,2],
                     header=1,
                     sep="\t",
                     names=[ "nSeq", "aa", "I2", "O1", "O2", "A5", "A6", "NaN"])
hr = hr.drop(["aa", "I2", "O1", "O2", "NaN"], 
             axis=1)
hr.head()

,nSeq,A5,A6
0,1,506,212
1,2,504,212
2,3,501,212
3,4,499,212
4,5,497,212


Wczytanie potrzebnych plików


In [27]:
images_params = pandas.read_csv(IAPS_filename, 
                                sep=";",
                                header=0,
                                decimal=",")
sounds_params = pandas.read_csv(IADS2_filename, 
                                sep=";", 
                                header=0,
                                decimal=",")

images_params.head()
sounds_params.head()

,Sound,Number,ValenceMean,ValenceSD,ArousalMean,ArousalSD
0,Cat,102,4.63,2.17,4.91,1.97
1,Panting,104,4.96,1.68,5.37,1.66
2,Puppy,105,2.88,2.14,6.4,2.13
3,Growl1,106,3.37,1.64,6.39,1.62
4,Dog,107,5.47,2.22,5.85,1.81


Zorientowanie się, że istnieją duplikaty w IAPS.
Usunięcie kolumny set (bo nie wiemy jak ją interpretować). 
Zamiana typu kolumny **IAPS** z *float* na *int*.
W przypadku tych samych obrazków, a znajdujących się w innych setach bierzemy średnią z parametrów.


In [28]:
print(f'Ilość duplikatów w parametrach obrazków'
      f' {images_params[images_params.duplicated(["IAPS"])].shape[0]}.')

images_params.drop(['set'], axis=1)
images_params.IAPS = images_params.IAPS.astype(int)
images_params = images_params.groupby('IAPS').agg({
                                                'ValenceMean': 'mean',
                                                'ValenceSD': 'mean',
                                                'ArousalMean': 'mean',
                                                'ArousalSD': 'mean',
                                                   }).reset_index()

Ilość duplikatów w parametrach obrazków 12.


In [29]:
print(f'Ilość duplikatów w parametrach dźwięków'
      f' {sounds_params[sounds_params.duplicated(["Number"])].shape[0]}.')

Ilość duplikatów w parametrach dźwięków 0.


Złączenie wyników badań i parametrów dźwięków i obrazków (Свекола это сука).
Usunięcie wierszy, w których $condition != inc$ i $widget != emospace1$
*df* - główny dataframe, na którym pracujemy

In [48]:
df = pandas.read_csv(df_filename,
                            sep="\t",
                            names=["person_id", "stimulus_id",
                                   "condition", "condition_details", 
                                   "sound", "image", "widget",
                                   "response", "response_time", "timestamp"])
df = df.loc[df['condition'] == 'inc'] \
    .loc[df['widget'] == 'emospace1']
df = df.join(sounds_params.set_index('Number'),
                           on="sound",
                           rsuffix='_sound')
df = df.join(images_params.set_index('IAPS'),
                           on='image',
                           rsuffix='_image')
df = df.astype({"response_time": float})
df.head()

(31, 19)

Usunięcie niepotrzebnych kolumn z dataframe'a.


In [31]:
df = df.drop(['person_id', 'stimulus_id', 'condition',
         'sound', 'image', 'widget',
         'response_time', 'timestamp', 'Sound'],
                axis=1)
df.head()

,condition_details,response,ValenceMean,ValenceSD,ArousalMean,ArousalSD,ValenceMean_image,ValenceSD_image,ArousalMean_image,ArousalSD_image
6,p+s-,"[0.0, -0.8871154785156252]",2.22,1.47,7.52,1.90,6.44,2.22,7.07,1.78
7,p-s+,"[-0.889205932617187, 0.0]",6.77,1.84,6.32,1.66,2.26,1.57,6.55,2.20
8,p+s-,"[-0.6720916748046881, -0.6839477539062505]",1.63,1.13,7.79,1.63,5.89,1.95,6.21,2.26
9,p+s-,"[-0.6523760986328123, 0.6662744140625002]",2.89,1.67,6.91,2.02,6.83,1.70,5.40,2.15
10,p-s+,"[-0.848992919921875, 0.0]",7.65,1.58,7.12,1.83,2.95,1.88,5.91,2.15


In [47]:
df['response'] = [list(map(lambda x: float(x), x.strip('[]').split(','))) for x in df['response']]

df.dtypes

(31, 10)


AttributeError: 'list' object has no attribute 'strip'

Dzielimy dataframe'a na tabelę gdzię obrazek jest pozytywny 
i dźwięk negatytwny (df_pi_ns) oraz na odwrót (df_ni_ps).
  

In [33]:
df_pi_ns = df.loc[df['condition_details'] == 'p+s-'] \
    .drop(['condition_details'], axis=1)
df_ni_ps = df.loc[df['condition_details'] == 'p-s+'] \
    .drop(['condition_details'], axis=1)
    
df_ni_ps.head()


,response,ValenceMean,ValenceSD,ArousalMean,ArousalSD,ValenceMean_image,ValenceSD_image,ArousalMean_image,ArousalSD_image
7,"[-0.889205932617187, 0.0]",6.77,1.84,6.32,1.66,2.26,1.570,6.55,2.20
10,"[-0.848992919921875, 0.0]",7.65,1.58,7.12,1.83,2.95,1.880,5.91,2.15
14,"[0.8944860839843755, -0.016984252929687502]",6.32,1.76,6.35,2.15,1.75,1.235,7.21,2.05
20,"[0.6604333496093752, 0.6413775634765618]",7.9,1.53,6.85,2.16,3.10,1.900,6.26,1.98
22,"[-0.8678228759765627, 0.0]",7,2.17,6.44,1.73,2.47,1.520,5.75,2.43


Stwierdzamy czy typ obrazka wpłynął na zadowolenie osoby badanej


In [51]:
sum(list(map(lambda x: 1 if x[0] >= 0 else 0 , df_pi_ns.loc[:,'response']))) / df_pi_ns.loc[:,'response'].size

0.4

In [57]:
sum(list(map(lambda x: 1 if x[0] < 0 else 0 , df_ni_ps.loc[:,'response']))) / df_ni_ps.loc[:,'response'].size

0.5625